# Statistics
## Run statistics

In [1]:
%autoreload 2 
import numpy as np 
from pianoq.misc.mplt import mplot, mimshow
from pianoq.simulations.abstract_quantum_scaling.qwfs_simulation import QWFSSimulation, QWFSResult
from pianoq.misc.mplc_writeup_imports import * 


In [97]:
N_modes = 256
s = QWFSSimulation(N=N_modes)

# algos = ['slsqp', "L-BFGS-B", 'simulated_annealing', 'genetic_algorithm', 'PSO']
algos = ['slsqp', "L-BFGS-B", 'simulated_annealing', 'genetic_algorithm', 'analytic']
algos = ['L-BFGS-B', 'analytic']
# configs = ['SLM1', 'SLM2', 'SLM3']
configs = ['SLM1', 'SLM1-only-T', 'SLM2', 'SLM2-simple', 'SLM2-simple-OPC', 'SLM3']
T_methods = ['unitary', 'gaus_iid']

N_tries = 2

note = 'long_with_analytic'
saveto_path = rf'G:\My Drive\Projects\MPLC\results\simulations\{tnow()}_qwfs_{note}.npz'
res = s.statistics(algos=algos, configs=configs, T_methods=T_methods, N_tries=N_tries, saveto_path=saveto_path)

# results = np.zeros((N_T_methods, N_configs, N_tries, N_algos))

try_no=0
try_no=1


## Show results 

In [31]:
print(np.pi/4)
print((np.pi/4)**2)

0.7853981633974483
0.6168502750680849


In [103]:
res.print()

---- SLM1 ----
-- unitary --
L-BFGS-B                  0.774+-0.01
analytic                  0.004+-0.00
-- gaus_iid --
L-BFGS-B                  0.737+-0.01
analytic                  0.001+-0.00

---- SLM1-only-T ----
-- unitary --
L-BFGS-B                  0.795+-0.01
analytic                  0.801+-0.00
-- gaus_iid --
L-BFGS-B                  0.734+-0.05
analytic                  0.737+-0.05

---- SLM2 ----
-- unitary --
L-BFGS-B                  0.629+-0.01
analytic                  0.004+-0.00
-- gaus_iid --
L-BFGS-B                  0.611+-0.00
analytic                  0.001+-0.00

---- SLM2-simple ----
-- unitary --
L-BFGS-B                  0.615+-0.01
analytic                  0.619+-0.01
-- gaus_iid --
L-BFGS-B                  0.535+-0.07
analytic                  0.564+-0.05

---- SLM2-simple-OPC ----
-- unitary --
L-BFGS-B                  0.962+-0.01
analytic                  1.000+-0.00
-- gaus_iid --
L-BFGS-B                  0.869+-0.10
analytic                  0.8

In [61]:
# comparing whether they find the same phases 
# mplot(best_phases[0, 2, 0, :3].T)
path = r"G:\My Drive\Projects\MPLC\results\simulations\2024_11_26_16_55_09_qwfs_long_with_analytic.npz"
res = QWFSResult(path)

In [95]:
res.show_violings()
plt.show(block=False)

In [26]:
# res.__dict__.keys()
# res.__dict__['arr_0'].item()['configs']
res.arr_0.item()['results'].mean()

0.542025838584121

## Understand SLM3 outputs 

In [2]:
path = r"G:\My Drive\Projects\MPLC\results\simulations\2024_11_26_16_55_09_qwfs_long_with_analytic.npz"
res = QWFSResult(path)

In [110]:
# results.shape == N_T_methods, N_configs, N_tries, N_algos
# best_phases.shape == N_T_methods, N_configs, N_tries, N_algos, self.N
try_no = np.random.randint(res.N_tries)
# try_no = 5
alg = 'L-BFGS-B'
config = 'SLM3'
# T_method = 'unitary'
T_method = 'gaus_iid'
alg_ind = np.where(res.algos == alg)[0]
conf_ind = np.where(res.configs == config)[0]
T_method_ind = np.where(res.T_methods == T_method)[0] 

T_ind = res.N_T_methods * try_no + T_method_ind 
T = res.Ts[T_ind].squeeze()
slm_phases = res.best_phases[T_method_ind, conf_ind, try_no, alg_ind].squeeze()
N = len(slm_phases)

sim = QWFSSimulation(N=N)
sim.T = T 
sim.slm_phases = np.exp(1j*slm_phases)
sim.config = config
v_out = sim.propagate()
I_out = np.abs(v_out)**2
I_in = np.abs(sim.v_in)**2
v_middle = sim.T.transpose() @ (sim.slm_phases * sim.v_in)
I_middle = np.abs(v_middle)**2
v_back_before_ft = sim.T @ v_middle
I_back_before_ft = np.abs(v_back_before_ft)**2
v_final_manual = np.fft.fft(v_back_before_ft) / np.sqrt(sim.N)
I_final_manual = np.abs(v_final_manual)**2
fig, ax = plt.subplots()
ax.plot(I_out, label='I_out')
ax.plot(I_middle, label='I_middle')
ax.set_ylim([0, 0.05])
ax.legend()
# fig.show()
print(f'{I_in.sum()=}')
print(f'{I_middle.sum()=}')
print(f'{I_back_before_ft.sum()=}')
print(f'{I_final_manual.sum()=}')
print(f'{I_out.sum()=}')

I_in.sum()=1.0
I_middle.sum()=1.61174585738503
I_back_before_ft.sum()=2.7545328929212376
I_final_manual.sum()=2.754532892921238
I_out.sum()=2.7545328929212385


In [74]:
mplot((np.abs(T@T.transpose())**2).sum(axis=0))

(<Figure size 640x480 with 1 Axes>, <AxesSubplot: >)

In [113]:
1.61**2

2.5921000000000003

In [54]:
print(alg_ind)
print(conf_ind)
print(T_method_ind)
res.T_methods

[0]
[5]
[0]


array(['unitary', 'gaus_iid'], dtype='<U8')

In [35]:
res.Ts.shape

(1200, 256, 256)

In [32]:
res.best_phases.shape

(2, 6, 600, 2, 256)

In [57]:
res.results.shape

(2, 6, 600, 2)

In [49]:
(np.abs(res.Ts[0, 0, :])**2).sum()

1.0000000000000002